Важным этапом обучения больших языковых моделей является их тонкая настройка для решения широкого круга задач; этот метод известен как тонкая настройка с учителем (supervised fine-tuning, SFT). Данный процесс помогает моделям стать более универсальными и способными работать при различных вариантах использования.

In [ ]:
# Установка необходимых библиотек
!pip install transformers datasets trl huggingface_hub -q

In [ ]:
import torch
from datasets import load_dataset
from IPython.core.display import display, HTML
from trl import SFTConfig, SFTTrainer, setup_chat_format
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Aвторизация на Hugging Face
from huggingface_hub import login

login()

# Шаблоны чатов

Шаблоны чатов (chat templates) необходимы для структурирования взаимодействия между языковыми моделями и пользователями. Понимание того, как правильно форматировать разговоры, имеет решающее значение для получения наилучших результатов модели. Рассмотрим, что такое шаблоны чата, почему они важны и как их использовать.

## Базовая модель vs. инструктивная модель

Базовая модель обучается на необработанных текстовых данных, чтобы предсказать следующий токен. Инструктивная модель настраивается специально для выполнения инструкций и участия в разговорах. Например, [SmolLM2-135M](https://huggingface.co/HuggingFaceTB/SmolLM2-135M) является базовой моделью, в то время как [SmolLM2-135M-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM2-135M-Instruct) – это ее вариант с дообучением на инструкциях.

Модели, настроенные на инструкции, обучаются следовать определенной структуре диалога, что делает их более подходящими для построения виртуальных ассистентов. Кроме того, инструктивные модели могут обрабатывать сложные взаимодействия, включая использование внешних инструментов, мультимодальный ввод данных и вызов функций.

Чтобы базовая модель стала инструктивной, нам нужно отформатировать промпты определенным образом, понятным модели. Вот тут-то и пригодятся шаблоны чата.

## Распространенные форматы шаблонов

Попробуем понять, как различные модели ожидают, что их диалоги будут отформатированы. Рассмотрим некоторые распространенные форматы шаблонов на простом примере диалога.

Для всех примеров будем использовать следующую структуру диалога:

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"},
    {"role": "assistant", "content": "Hi! How can I help you today?"},
    {"role": "user", "content": "What's the weather?"},
]

Это шаблон ChatML, используемый в таких моделях, как SmolLM2 и Qwen 2:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct")
chat = [
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "I'd like to show off how chat templating works!"},
]
after_formating = tokenizer.apply_chat_template(chat, tokenize=False)
print(after_formating)

Так используется шаблон Mistral:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
chat = [
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "I'd like to show off how chat templating works!"},
]
after_formating = tokenizer.apply_chat_template(chat, tokenize=False)
print(after_formating)

Основные различия между этими форматами следующие:

1. Обработка системных сообщений
- Llama 2 оборачивает системные сообщения в теги `<<SYS>>`
- В Llama 3 используются теги `<|system|>` и `</s>`
- Mistral включает системное сообщение в первый промпт
- Qwen использует явную системную роль с тегами `<|im_start|>`
- ChatGPT использует префикс `SYSTEM:`

2. Границы сообщений
- В Llama 2 используются теги `[INST]` и `[/INST]`
- В Llama 3 используются теги, относящиеся к конкретной роли (`<|system|>`, `<|user|>`, `<|assistant|>`), с окончаниями `</s>`
- Mistral использует теги `[INST]` и `[/INST]` с `<s>` и `</s>`

3. Специальные токены
- Llama 2 использует `<s>` и `</s>` для обозначения границ разговора
- Llama 3 использует `</s>` для завершения каждого сообщения
- Mistral использует `<s>` и `</s>` для обозначения границ
- Qwen использует начальные и конечные токены, зависящие от конкретной роли

## Расширенные функции

Шаблоны чата могут работать с более сложными сценариями, выходящими за рамки простого общения, включая:
- Использование внешних инструментов или API
- Мультимодальный ввод для обработки изображений, аудио
- Вызов функции для выполнения структурированного алгоритма
- Контекст из нескольких реплик для ведения истории диалога

Для мультимодальных диалогов шаблоны чата могут включать ссылки на изображения или изображения в кодировке base64:

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful vision assistant that can analyze images.",
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What's in this image?"},
            {"type": "image", "image_url": "https://example.com/image.jpg"},
        ],
    },
]

Пример шаблона чата с использованием внешнего инструмента:

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are an AI assistant that can use tools. Available tools: calculator, weather_api",
    },
    {"role": "user", "content": "What's 123 * 456 and is it raining in Paris?"},
    {
        "role": "assistant",
        "content": "Let me help you with that.",
        "tool_calls": [
            {
                "tool": "calculator",
                "parameters": {"operation": "multiply", "x": 123, "y": 456},
            },
            {"tool": "weather_api", "parameters": {"city": "Paris", "country": "France"}},
        ],
    },
    {"role": "tool", "tool_name": "calculator", "content": "56088"},
    {
        "role": "tool",
        "tool_name": "weather_api",
        "content": "{'condition': 'rain', 'temperature': 15}",
    },
]

## Общие принципы

При работе с шаблонами чата требуется следовать нескольким основным рекомендациям.
1. Единообразное форматирование: на протяжении всего проекта дожен использоваться один и тот же шаблон чата
2. Четкое определение ролей: роли (`system`, `user`, `assistant`, `tool`) должны быть указаны для каждого сообщения
3. Управление контекстом: необходимо помнить об ограничениях на количество токенов при ведении истории разговоров
4. Обработка ошибок: требуется тщательно продумать обработку ошибок для вызовов внешних инструментов и для мультимодальных входных данных

## Практическое упражнение

Попрактикуемся в реализации шаблонов чата на реальном примере.

### Шаблон чата SmolLM2

In [ ]:
# Определим модель и токенизатор

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [ ]:
# Зададим сообщение для SmolLM2

messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {
        "role": "assistant",
        "content": "I'm doing well, thank you! How can I assist you today?",
    },
]

### Применение шаблона чата без токенизации

Токенизатор представляет диалог в виде строки со специальными токенами, описывающими роль пользователя и ассистента.

In [ ]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False)

print("Conversation with template:", input_text)

### Декодирование диалога

Обратите внимание, что диалог представлен так же, как и выше, но с дополнительным сообщением ассистента.

In [ ]:
input_text = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True
)

print("Conversation decoded:", tokenizer.decode(token_ids=input_text))

### Токенизация диалога

Токенизатор сопоставляет токенам диалога и специальным токенам идентификаторы из словаря модели.

In [ ]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

print("Conversation tokenized:", input_text)

### Преобразование данных для SFT

Возьмем набор данных [smoltalk](https://huggingface.co/datasets/HuggingFaceTB/smoltalk) из Hugging Face и предобработаем его для тонкой настройки с учителем.

In [ ]:
display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/HuggingFaceTB/smoltalk/embed/viewer/all/train?row=0"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

In [ ]:
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
ds["train"][0]

Преобразуем каждое сообщение в формат чата с помощью токенизатора.

In [ ]:
def process_dataset(sample):
    sample["no_tokenization"] = tokenizer.apply_chat_template(sample["messages"], tokenize=False, add_generation_prompt=True)
    sample['tokenized'] = tokenizer.apply_chat_template(sample["messages"], add_generation_prompt=True)
    return sample

In [ ]:
ds = ds.map(process_dataset)

In [ ]:
for k,v in ds["train"][0].items():
  print(f"{k}:\n{v}\n")

Мы узнали, как применять шаблоны чата к различным моделям. Структурируя взаимодействия с помощью шаблонов чата, мы можем гарантировать, что языковые модели будут выдавать согласованные и контекстуально значимые ответы.

# Тонкая настройка с учителем

Данный процесс используется для адаптации предобученных базовых языковых моделей для выполнения инструкций, ведения диалога и использования определенных выходных форматов. Несмотря на то, что предварительно обученные модели обладают впечатляющими общими возможностями, SFT помогает преобразовать их в модели, похожие на виртуальных ассистентов, которые могут лучше понимать запросы пользователя и реагировать на них. Обычно это достигается путем обучения на основе наборов данных с написанными человеком разговорами и инструкциями.

## Когда и для чего использовать SFT

SFT позволяет точно контролировать структуру выходных данных модели. Это особенно ценно, когда нужно, чтобы модель:
1. Генерировала ответы в определенном формате шаблона чата
2. Строго следовала схемам вывода
3. Поддерживала единообразный стиль ответов

SFT помогает привести модель в соответствие с требованиями конкретной предметной области посредством:

1. Обучения терминологии и концепциям предметной области
2. Предписания профессиональных стандартов
3. Надлежащей обработке технических запросов
4. Следования отраслевым рекомендациям

Для тонкой настройки с учителем требуется набор данных из пар ввода-вывода. Каждая пара должна состоять из:
1. Входного промпта
2. Ожидаемого ответа модели
3. Любой дополнительной информации или метаданных

Качество обучающих данных имеет решающее значение для успешной тонкой настройки.

## Конфигурация обучения

Успех тонкой настройки также во многом зависит от правильного выбора гиперпараметров обучения.
1. Параметры, определяющие продолжительность обучения
- `num_train_epochs`: контролирует общую продолжительность обучения
- `max_steps`: аьтернатива эпохам, устанавливает максимальное количество этапов обучения
- большее количество эпох позволяет лучше обучаться, но сопряжено с риском переобучения

2. Параметры, определяющие размер батча
- `per_device_train_batch_size`: от него зависит использование памяти и стабильность обучения
- `gradient_accumulation_steps`: позволяет обновлять веса не после каждого батча, а после заданного количества этапов
- больший размер батча обеспечивает более стабильное обучение, но требует больше памяти

3. Параметры, определяющие скорость обучения
- `learning_rate`: контролирует силу обновлений весов
- `warmup_ratio`: определяет долю обучающей выборки, которая используется для увеличения скорости обучения
- слишком высокое значение может привести к нестабильности, слишком низкое — к медленному обучению

4. Параметры, которые позволяют отслеживать процесс обучения
- `logging_steps`: частота вывода метрики
- `eval_steps`: как часто проводить оценку на основе валидационных данных
- `save_steps`: частота сохранения контрольной точки модели

Стоит начать с базовых значений и корректировать их на основе мониторинга:
- 1-3 эпохи
- небольшой размер батча

Следует внимательно следить за показателями на валидационной выборке и корректировать скорость обучения, если оно нестабильно.

## Отслеживание процесса обучения

Значение **функции потерь (loss function)** обычно изменяется в три этапа:

- Начальный резкий спад: быстрая адаптация к новому распределению данных
- Постепенная стабилизация: скорость обучения замедляется по мере доработки модели
- Сходимость: значение функции потерь стабилизируется, что указывает на завершение обучения

На приведенном графике показан типичный процесс обучения. Обратите внимание, что значение функции потерь на обучении и на валидации снижается сначала резко, а затем постепенно. Эта закономерность указывает на то, что модель эффективно обучается, сохраняя способность к обобщению.

<center><img src="https://i.postimg.cc/vBxgYCDv/loss.png" width="600"></center>

Эффективный мониторинг предполагает отслеживание количественных показателей и оценку качественных показателей, таких как:

- значение функции потерь на обучающей выборке (training loss)
- значение функции потерь на валидационной выборке (validation loss)
- изменение скорости обучения (learning rate progression)

По мере прохождения обучения график функции потерь должен постепенно стабилизироваться. Ключевым показателем правильного обучения является небольшой разрыв между значением функции потерь при обучении и при валидации. Это позволяет предположить, что модель изучает обобщаемые закономерности, а не запоминает конкретные примеры. Абсолютные значения функции потерь будут варьироваться в зависимости от вашей задачи и набора данных.

Необходимо следить за несколькими предупреждающими знаками во время обучения:
1. Значение функции потерь при валидации увеличивается, а при обучении уменьшается (переобучение)
2. Значение функции потерь незначительно увеличивается (недостаточное обучение)
3. Чрезвычайно низкое значение функции потерь (потенциальное запоминание)
4. Несогласованное форматирование выходных данных (проблемы с усвоением шаблонов)

Если значение функции потерь при валидации уменьшается значительно медленнее, чем при обучении, то  модель, скорее всего, подстраивается под данные обучения. Чтобы это исправить, можно:
- Сократить количество этапов обучения
- Увеличить размер набора данных
- Проверить качество и разнообразие набора данных

<center><img src="https://i.postimg.cc/CxWVytT0/loss1.png" width="600"></center>

Если значения функции потерь не показывают существенного улучшения, возможно, модель:

- Обучается слишком медленно (попробуйте увеличить скорость обучения)
- Возникают трудности с задачей (проверьте качество данных и сложность задачи)
- Возникают архитектурные ограничения (рассмотрите другую модель)

<center><img src="https://i.postimg.cc/RZJjTR3D/loss2.png" width="600"></center>

Чрезвычайно низкое значение функции потерь может свидетельствовать скорее о запоминании, чем об обучении. Это особенно важно, если:

- Модель плохо работает на новых, похожих примерах
- В выходных данных отсутствует разнообразие
- Ответы слишком похожи на обучающие примеры

<center><img src="https://i.postimg.cc/ZqFsfDJV/loss3.png" width="600"></center>

## Практическое упражнение

Рассмотрим, как осуществить тонкую настройку модели [SmolLM2-135M](https://huggingface.co/HuggingFaceTB/SmolLM2-135M) с помощью [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) из библиотеки [TRL](https://huggingface.co/docs/trl/index).

<center><img src="https://i.postimg.cc/KjrnbCsH/trl-banner-dark.png" width="800"></center>

In [ ]:
# Определим модель и токенизатор
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

### Генерация с помощью базовой модели

In [ ]:
# Зададим промпт
prompt = "Hello! How are you?"

# Отформатируем его
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Сгенерируем ответ
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Подготовка данных

Загрузим набор данных и отформатируем его для обучения. Набор данных должен содержать пары ввода-вывода, где каждый ввод является промптом, а вывод — ожидаемым ответом модели.

Библиотека TRL форматирует входные сообщения на основе шаблонов чата модели. Они должны быть представлены в виде списка словарей с ключами `role` и `content`.

In [ ]:
# Загрузим датасет
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
ds["train"][1]

### Определение параметров для SFTTrainer

В SFTTrainer настраиваются различные параметры, которые управляют процессом обучения. К ним относится количество эпох обучения, размер батча, скорость обучения и способ оценки.

In [ ]:
# Установка параметров
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # корректирется в зависимости от размера набора данных и желаемой продолжительности обучения
    per_device_train_batch_size=4,  # устанавливается в соответствии с объемом памяти графического процессора
    learning_rate=5e-5,  # отправная точка для тонкой настройки
    logging_steps=10,  # частота отслеживания показателей обучения
    save_steps=100,  # частота сохранения модели
    logging_strategy="steps",  # оценивает модель через регулярные промежутки времени
    eval_steps=50,  # частота валидации
    report_to="none"
)

# Инициализация
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    processing_class=tokenizer,
    eval_dataset=ds["test"],
)

### Обучение модели

In [ ]:
trainer.train()

### Генерация с помощью дообученой модели

In [ ]:
# Зададим тот же промт
prompt = "Hello! How are you?"

# Отформатируем его
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Сгенерируем ответ
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Мы рассмотрели пошаговое руководство по тонкой настройке модели SmolLM2-135M с помощью SFTTrainer. Эти этапы позволили эффективно адаптировать модель для выполнения конкретных задач.

# Оценка LLM

После тонкой настройки модели с помощью SFT мы должны оценить ее с помощью ряда стандартных бенчмарков.

## Автоматическая оценка

Автоматические бенчмарки служат стандартизированными инструментами для оценки языковых моделей с учетом различных задач и возможностей. Хотя они являются полезной отправной точкой для понимания эффективности моделей, важно понимать, что они представляют собой лишь часть комплексной стратегии оценки.

Автоматические бенчмарки обычно состоят из тщательно отобранных наборов данных с определенными задачами и оценочными метриками. Они направлены на оценку различных аспектов возможностей модели, от базового понимания языка до сложных рассуждений. Ключевым преимуществом использования автоматических бенчмарков является их стандартизация — они позволяют проводить последовательное сравнение различных моделей и обеспечивают воспроизводимость результатов.

Однако важно понимать, что результаты не всегда напрямую влияют на эффективность в реальном мире. Модель, которая отлично справляется с академическими бенчмарками, может по-прежнему испытывать трудности с приложениями в конкретной предметной области или практическим использованием.

### Оценка общих знаний о мире

[MMLU](https://huggingface.co/datasets/cais/mmlu) (Massive Multitask Language Understanding) проверяет знания по 57 предметам, от естественных до гуманитарных. Хотя бенчмарк и является разносторонним, он может не отражать глубину знаний, необходимых для конкретных областей. Генерация ответа на вопрос оценивает склонность модели воспроизводить распространенные заблуждения, хотя и не может охватить все формы дезинформации.

In [ ]:
mmlu = load_dataset(path="cais/mmlu", name="machine_learning")

In [ ]:
mmlu["validation"][0]

### Оценка способности к рассуждению

[BBH](https://huggingface.co/datasets/lukaemon/bbh) (Big Bench Hard) и [GSM8K](https://huggingface.co/datasets/openai/gsm8k) (Grade School Math 8K) ориентированы на сложные логические задачи. BBH тестирует логическое мышление и планирование, в то время как GSM8K ориентирован на решение математических задач. Эти бенчмарки помогают оценить аналитические способности, но могут не отражать нюансы мышления, необходимые в реальных сценариях.

In [ ]:
bbh = load_dataset(path="lukaemon/bbh", name="temporal_sequences")

In [ ]:
bbh["test"][0]

In [ ]:
gsm8k = load_dataset(path="openai/gsm8k", name="main")

In [ ]:
gsm8k["train"][1]

### Бенчмарки, относящиеся к конкретной предметной области

Бенчмарк [MATH](https://huggingface.co/papers/2103.03874) — это еще один важный инструмент оценки математического мышления. Он состоит из 12 500 задач из соревнований по математике, охватывающих алгебру, геометрию, теорию чисел, теорию вероятностей и многое другое. Что делает бенчмарк особенно сложным, так это то, что он требует многоэтапного рассуждения, понимания формальных математических обозначений и умения генерировать пошаговые решения. В отличие от простых арифметических задач, математические задачи часто требуют сложных стратегий решения задач и применения математических концепций.

In [ ]:
math = load_dataset(path="qwedsacf/competition_math")

In [ ]:
math["train"][2]

[HumanEval Benchmark](https://huggingface.co/datasets/openai/openai_humaneval) — это набор данных, состоящий из 164 задач по программированию. Он проверяет способность модели генерировать функционально корректный код на Python, который решает поставленные задачи программирования. Что делает HumanEval особенно ценным, так это то, что он оценивает как возможности генерации кода, так и функциональную корректность на основе реального выполнения тестовых примеров, а не просто поверхностного сходства с эталонными решениями. Задачи варьируются от базовых манипуляций со строками до более сложных алгоритмов и структур данных.

In [ ]:
human_eval = load_dataset(path="openai/openai_humaneval")

In [ ]:
for k,v in human_eval["test"][0].items():
  print(f"{k}: {v}")

[Alpaca Eval](https://tatsu-lab.github.io/alpaca_eval/) — это автоматизированная система оценки, разработанная для оценки инструктивных языковых моделей. Она использует GPT-4 в качестве критерия для оценки результатов генерации по различным параметрам, включая полезность (helpfulness), честность (honesty) и безвредность (harmlessness). Фреймворк включает в себя набор данных из 805 тщательно отобранных запросов. Финальная метрика показывает, как часто результаты оцениваемой модели оказались предпочтительнее результатов эталонной модели. Что делает Alpaca Eval особенно полезным, так это его способность предоставлять согласованные, масштабируемые оценки без использования аннотаторов-людей, при этом сохраняя нюансы производительности модели, которые могут быть упущены при использовании традиционных показателей.

## Альтернативные подходы к оценке

Были разработаны альтернативные методы оценки, чтобы устранить ограничения стандартных бенчмарков.

### Модель-арбитр (LLM-as-Judge)

Использование одной языковой модели для оценки результатов другой становится все более популярным. Этот подход может обеспечить более детальную обратную связь, чем традиционные показатели, хотя и имеет свои недостатки и ограничения.

### Арены для оценки

Арены для оценки как [Chatbot Arena](https://lmarena.ai/) предлагают уникальный подход к оценке LLM с помощью краудсорсинговой обратной связи. На этих платформах пользователи участвуют в анонимных “битвах” между двумя LLM, задавая вопросы и голосуя за то, какая модель дает лучшие ответы. Этот подход отражает реальные паттерны использования и предпочтения пользователей с помощью разнообразных, сложных вопросов, а исследования показывают сильное соответствие между голосованиями из краудсорсинга и экспертными оценками. Несмотря на свою мощь, эти платформы имеют свои ограничения, в том числе потенциальную предвзятость в отношении базы пользователей, неравномерное распределение запросов и основное внимание на полезности, а не на безопасность.

### Пользовательские бенчмарки

Организации часто разрабатывают внутренние наборы тестов, адаптированные к их конкретным потребностям и вариантам использования. Они могут включать вопросы на знание предметной области или сценарии оценки, которые отражают реальные условия использования.

# Обобщение

Мы изучили основные компоненты тонкой настройки языковых моделей:
1. **Шаблоны чатов** предоставляют структуру для моделирования взаимодействий, обеспечивая согласованность и адекватность ответов благодаря стандартизированному форматированию.
2. **Тонкая настройка с учителем (SFT)** позволяет адаптировать предварительно обученные модели к конкретным задачам, сохраняя при этом их базовые знания.
3. **Оценка** помогает измерить и подтвердить эффективность тонкой настройки с помощью различных показателей и бенчмарков.